In [58]:
# !pip install torch
import pandas as pd
import numpy as np
import warnings  # >>> 3shna lw 3awz ashyl warning zahr fe cell wla haga
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [59]:
df = pd.read_csv(r"/content/HousingData.csv")
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [60]:
warnings.filterwarnings("ignore")

x = df.drop(["MEDV"],1)
y = df["MEDV"]

In [62]:
null_columns = x.columns[x.isnull().any()]
null_rows = x[x.isnull().any(axis=1)]

x = x.dropna()
y = y.drop(null_rows.index)  # Remove corresponding target values

x = x.reset_index(drop=True)
y = y.reset_index(drop=True)

In [63]:
x.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [64]:
scaler = StandardScaler()
columns = x.columns    # >> 3shan lma b3ml fit transform byzharly arrays w msh bkon 3arf esm elcolumns
x = scaler.fit_transform(x)
x = pd.DataFrame(x,columns = columns)

In [65]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3)
print("x_train shape ",x_train.shape)
print("y_train shape ",y_train.shape)
print("x_test shape ",x_test.shape)
print("y_test shape ",y_test.shape)

x_train shape  (275, 13)
y_train shape  (275,)
x_test shape  (119, 13)
y_test shape  (119,)


In [66]:
import torch
from torch import nn

class data(torch.utils.data.Dataset):
    def __init__(self,x,y):
        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y)
    def  __len__(self):
        return len(self.x)
    def __getitem__(self,i):
        return self.x[i] , self.y[i]


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(13,32),
            nn.Sigmoid(),
            nn.Linear(32,1))
    def forward(self,inputs):
        return self.layers(inputs)


NN = NeuralNetwork()
train_dataset = data(x_train.values,y_train.values)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=8)


optimizer = torch.optim.SGD(NN.parameters(),lr=1e-4)
cost = nn.MSELoss()
epochs = 100

for epoch in range(epochs):
    print("epoch : ",epoch)
    total_training_loss = 0
    for i,data in enumerate(train_loader,0):
        inputs,targets = data
        inputs,targets = inputs.float(),targets.float()
        targets = targets.reshape((-1,1))
        optimizer.zero_grad()
        preds = NN.forward(inputs)
        loss = cost(preds,targets)
        total_training_loss+= loss.item()
        loss.backward()
        optimizer.step()
    print("total_training_loss : ",total_training_loss)


epoch :  0
total_training_loss :  19583.59262084961
epoch :  1
total_training_loss :  17577.011138916016
epoch :  2
total_training_loss :  15794.392135620117
epoch :  3
total_training_loss :  14202.298446655273
epoch :  4
total_training_loss :  12774.420913696289
epoch :  5
total_training_loss :  11489.921966552734
epoch :  6
total_training_loss :  10332.199645996094
epoch :  7
total_training_loss :  9287.923690795898
epoch :  8
total_training_loss :  8346.263137817383
epoch :  9
total_training_loss :  7498.247131347656
epoch :  10
total_training_loss :  6736.242607116699
epoch :  11
total_training_loss :  6053.538455963135
epoch :  12
total_training_loss :  5444.028072357178
epoch :  13
total_training_loss :  4901.986804962158
epoch :  14
total_training_loss :  4421.930328369141
epoch :  15
total_training_loss :  3998.5481357574463
epoch :  16
total_training_loss :  3626.678909301758
epoch :  17
total_training_loss :  3301.327723503113
epoch :  18
total_training_loss :  3017.699837684

In [67]:
preds = NN.forward(torch.from_numpy(x_test.values).float())

In [68]:
preds = preds.detach().numpy().reshape(-1,)


res = pd.DataFrame()
res["Actual"] = y_test.values
res['preds'] = preds
res

,Actual,preds
0,24.7,28.597282
1,10.4,16.209854
2,18.8,19.072437
3,19.2,21.348007
4,23.8,25.344490
...,...,...
114,14.1,17.325022
115,13.1,15.728938
116,20.2,17.731339
117,32.0,31.255058
